# CECS 229 Final Exam Part 1

#### Due Date: 

Tuesday, 5/10 @ 11:59 PM

#### Worth:

15% of the Final Exam grade category

#### Submission Instructions:

To receive credit for this assignment you must submit the following by the due date:

1. **To the BB Dropbox Folder:** this completed .ipynb file

2. **To CodePost:** this file converted to a Python script named `final.py`

----------------------------------------------------------

#### Background:

In this programming assignment, you will be responsible for implementing a solver for the system of linear equations $A \overrightarrow{x} = \overrightarrow{b}$ where 

* $A$ is an $n \times n$ matrix whose columns are linearly independent
* $\overrightarrow{x} \in \mathbb{R}^n$
* $\overrightarrow{b} \in \mathbb{R}^n$

To implement the solver, you must apply the following theorem:

**THM |  QR-Factorization** 

If $A \in \mathbb{F}_{m \times n}$ matrix with linearly independent columns $\overrightarrow{a_1}, \overrightarrow{a_2}, \dots \overrightarrow{a_n}$, then there exists,

1. an $m \times n$ matrix $Q$ whose columns $\overrightarrow{u_1}, \overrightarrow{u_2}, \dots, \overrightarrow{u_n}$ are orthonormal, and 
2. an $n \times n$ matrix $R$ that is upper triangular and whose entries are defined by,
$r_{ij} = \begin{cases}
\langle \overrightarrow{u_i}, \overrightarrow{a_j} \rangle & \text{ for } i \leq j \\
0 & \text{ for } i > j \\
\end{cases}$

such that $A = QR$.  This referred to as the QR factorization (or decomposition) of matrix $A$.


To find matrices $Q$ and $R$ from the QR Factorization Theorem, we apply Gram-Schimdt process to the columns of $A$.  Then,

* the columns of $Q$ will be the orthonormal vectors $\overrightarrow{u_1}, \overrightarrow{u_2}, \dots, \overrightarrow{u_n}$ returned by the Gram Schimdt process, and
* the entries $r_{ij}$ of $R$ will be computed using each column $\overrightarrow{u_i}$ as defined in the theorem.

Luckily, you do not need to implement this process.  A Python library called `numpy` contains a module called `linalg` with a function called `qr()` that returns the matrices $Q$ and $R$ in the QR factorization of a matrix $A$.  Try running the following cell to see how it works.

In [5]:
import numpy as np
import math
class Vec:
    def __init__(self, contents = []):
        """
        Constructor defaults to empty vector
        INPUT: list of elements to initialize a vector object, defaults to empty list
        """
        self.elements = contents
        return 
    
    def __abs__(self):
        """
        Overloads the built-in function abs(v)
        returns the Euclidean norm of vector v
        """
        return math.sqrt(sum([i**2 for i in self.elements]))

        
    def __add__(self, other):
        """Overloads the + operator to support Vec + Vec
         raises ValueError if vectors are not same length
        """
        if len(self.elements) != len(other.elements):
            raise ValueError
        else:
            return Vec([x + y for x, y in zip(self.elements, other.elements)])


    
    def __sub__(self, other):
        """
        Overloads the - operator to support Vec - Vec
        Raises a ValueError if the lengths of both Vec objects are not the same
        """
        if len(self.elements) != len(other.elements):
            raise ValueError
        else:
            return Vec([x-y for x,y in zip(self.elements,other.elements)])

    
    
    def __mul__(self, other):
        """Overloads the * operator to support 
            - Vec * Vec (dot product) raises ValueError if vectors are not same length in the case of dot product
            - Vec * float (component-wise product)
            - Vec * int (component-wise product)
        """
        if type(other) == Vec:
            if len(self.elements) != len(other.elements):
                raise ValueError
            elif len(self.elements) == len(other.elements):
                return (sum([x*y for x,y in zip(self.elements,other.elements)]))
        elif type(other) == float or type(other) == int: #scalar-vector multiplication
            return Vec([i*other for i in self.elements])
            
    
    def __rmul__(self, other):
        """Overloads the * operation to support 
            - float * Vec
            - int * Vec
        """
        return Vec([i * other for i in self.elements])
    

    
    def __str__(self):
        """returns string representation of this Vec object"""
        return str(self.elements) # does NOT need further implementation

    def norm(self,p):
        y = 0
        for i in self.elements:
            x=abs(i)**p
            y+=x
        return y**(1/p)

class Matrix:
    def __init__(self,rows = []):  
        if len(rows) != 0:
            cols = len(rows[0])
            if cols == 0:
                raise ValueError
            for row in rows:
                if len(row) != cols:
                    raise ValueError
                for value in row:
                    if not isinstance(value,(int,float)):
                        raise ValueError
            self.rows = rows
        else:
            self.rows = []
        
    def __add__(self, other):
        if ((len(self.rows) != len(other.rows)) or (len(self.rows[0])!= len(other.rows[0]))):
            raise ValueError("Matrix must be of the same size.")
        return Matrix([[self.rows[i][j] + other.rows[i][j] for j in range((len(self.rows[0])))] for i in range(len(self.rows))])
    
    def __sub__(self, other):
        if ((len(self.rows) != len(other.rows)) or (len(self.rows[0])!= len(other.rows[0]))):
            raise ValueError("Matrix must be of the same size.")
        return Matrix([[self.rows[i][j] - other.rows[i][j] for j in range((len(self.rows[0])))] for i in range(len(self.rows))])
        
    def __mul__(self, other):  
        if type(other) == float or type(other == int):
            return Matrix([[item * other for item in row] for row in self.rows])
        elif type(other) == Matrix:
            if ((len(self.rows[0]) != len(other.rows))):
                raise ValueError("Matrix must be the same size")
            return Matrix([
                [sum(row[i] * column[i] for i in range(len(row)))for column in [[row[i] for row in other.rows] 
                for i in range(len(other.rows[0]))]]
                for row in self.rows
                ]
            )
        elif type(other) == Vec:
            if len(self.rows[0]) != len(other.elements):
                raise ValueError("Vector must be of length of columns.")
            return np.dot(self,other)

        else:
            raise TypeError("ERROR: Unsupported Type.")
    
    def __rmul__(self, other):  
        if type(other) == float or type(other) == int:
           return Matrix([[item * other for item in row] for row in self.rows])
        else:
            raise TypeError("ERROR: Unsupported Type.")
    
    def __str__(self):
        """prints the rows and columns in matrix form """
        return '\n'.join(str(e) for e in self.row)
    
    def __eq__(self, other):
        """overloads the == operator to return True if 
        two Matrix objects have the same row space and column space"""
        this_rows = self.row_space()
        other_rows = other.row_space()
        this_cols = self.col_space()
        other_cols = other.col_space()
        return this_rows == other_rows and this_cols == other_cols

    def __req__(self, other):
        """overloads the == operator to return True if 
        two Matrix objects have the same row space and column space"""
        this_rows = self.row_space()
        other_rows = other.row_space()
        this_cols = self.col_space()
        other_cols = other.col_space()
        return this_rows == other_rows and this_cols == other_cols
    
    def order(self):
        return (self.row, self.row[0])
    
    def set_col(self, j, u):
        if len(u) != len(self.rows):
            raise ValueError("Incompatible column length.")    
        else:
            self.rows  = [self.rows[i][0:j-1] + [u[i]] + self.rows[i][j:] for i in range(len(self.rows))]
        return self
        
    def set_row(self,i, v):
        if len(v) != len(self.rows[0]):
            raise ValueError("Incompatible row length.")    
        else:
            self.rows[i-1] = v
        return self
    
    def set_entry(self,i, j, x):
        self.rows[i-1][j-1] = x
        return self
    
    def get_col(self,j):
        return [self.rows[i][j-1] for i  in range(len(self.rows))]
    
    def get_row(self,i):
        return self.rows[i-1]
    
    def get_entry(self, i ,j):
        return self.rows[i-1][j-1]
    
    def col_space(self):
        return[[row[i] for row in self.rows] for i in range(len(self.rows[0]))]
    
    def row_space(self):
        return self.rows
    
    def get_diag(self, k):
        if k == 0:
            return [self.rows[i][i] for i in range(len(self.rows))]
        if k > 0:
            return[self.rows[i][i+(abs(k))] for i in range(len(self.rows)-abs(k))]
        if k < 0:
             return[self.rows[i+(abs(k))][i] for i in range(len(self.rows)-abs(k))]
         
    def rank(self):
        r1 = len(self.rows)
        rank = len(self.rows[0])
        for r in range(0,rank,1):
            if self.rows[r][r] != 0:
                for c in range(0,r1,1):
                    if c != r:
                        m = (self.rows[c][r] / self.rows[r][r])
                        for i in range(rank):
                            self.rows[c][i] -= (m * self.rows[r][i])
            else:
                shrink = True
                for i in range(r+1,r1,1):
                    if self.rows[i][r] != 0:
                        for b in range(c):
                            temp = self.rows[r][b]
                            self.rows[r][b] = self.rows[i][b]
                            self.rows[i][b] = temp
                        shrink = False
                        break
                if shrink:
                    rank -=1
                    for i in range(0,r1,1):
                        self.rows[i][r] = self.rows[i][rank]
                r-=1
        return rank     

In [10]:
import numpy as np

A = [[1, 1, 0], [0, 1, 1], [1, 0, 1]]
Q, R = np.linalg.qr(A)

print("Q =\n", Q)
print("\nR =\n", R)

print("\n\nOriginal A =\n", Matrix(A))
print("\nQ * R =\n", Matrix(Q) * Matrix(R))

Q =
 [[-0.70710678 -0.40824829 -0.57735027]
 [-0.         -0.81649658  0.57735027]
 [-0.70710678  0.40824829  0.57735027]]

R =
 [[-1.41421356 -0.70710678 -0.70710678]
 [ 0.         -1.22474487 -0.40824829]
 [ 0.          0.          1.15470054]]


Original A =
 [1, 1, 0]
[0, 1, 1]
[1, 0, 1]


AttributeError: type object 'int' has no attribute 'row_space'

-----------------------------------------------------

#### Your Task:

Assuming $A \in \mathbb{R}_{n \times n}$ is a `Matrix` object, and $\overrightarrow{b} \in \mathbb{R}^n$ is a `Vec` object, implement a function `solve_qr(A, b)` that uses the QR-factorization of $A$ to compute and return the solution to the system $A\overrightarrow{x} =  \overrightarrow{b}$.  

Hints:

1.  If $A = QR$, then $A\overrightarrow{x} = \overrightarrow{b}$ becomes $QR\overrightarrow{x} = \overrightarrow{b}$.  What happens if we multiply both sides of the equation by the transpose of $Q$? i.e., What does $Q^tQR\overrightarrow{x} = Q^t\overrightarrow{b}$ simplify to?
2.  Part of your code for the solvers in CA #6 will come in handy.

In [ ]:
import numpy as np
def solve_qr(A, b):
    # todo
    pass

In [ ]:
"""TESTER CELL"""

A = Matrix([[2, -2, 18], [2, 1, 0], [1, 2, 0]])
x_true = Vec([3, 4, 5])
b = A * x_true
print("Expected:", x_true)

x = solve_qr(A, b)
print("Returned:", x)